<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/TurboExpanderCompressor_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TurboExpanderCompressor Model Example

This notebook demonstrates how to use the **TurboExpanderCompressor** model in NeqSim using Python. This model simulates a coupled turbo-expander and compressor unit, where the power generated by the expander drives the compressor.

## Mathematical Basis

The expander and compressor share a shaft speed that is iteratively adjusted until the expander power balances the compressor power plus bearing losses.

Key features include:
- **Isentropic Flash**: Calculates enthalpy drop across the expander.
- **Efficiency Corrections**: Applies corrections based on Velocity Ratio (UC) and Flow Coefficient (Q/N).
- **Power Balance**: Solves for the shaft speed where $W_{expander} = W_{compressor} + W_{bearing}$.

For more details, refer to the

[TurboExpanderCompressor Model Documentation](https://github.com/equinor/neqsim/blob/master/docs/turboexpander_compressor_model.md).

and

[Tutorial on cryogeneic turboexpanders](https://oaktrust.library.tamu.edu/bitstream/handle/1969.1/163246/t33-15.pdf)

In [148]:
# Install NeqSim
# If running in Google Colab or a new environment, uncomment the line below
!pip install neqsim -q

In [149]:
import neqsim
from neqsim import jneqsim
from neqsim.thermo import fluid
from neqsim.process import stream

# 1. Create the Fluid
# Create a thermodynamic system using SRK-EoS
fluid = fluid('srk')
fluid.addComponent("methane", 90.0)
fluid.addComponent("ethane", 5.0)
fluid.addComponent("propane", 5.0)
fluid.addComponent("n-butane", 5.0)
fluid.setMixingRule("classic")

# 2. Create the Inlet Stream
inlet_stream = jneqsim.process.equipment.stream.Stream("Inlet Stream", fluid)
inlet_stream.setFlowRate(15.0, "MSm3/day")
inlet_stream.setTemperature(0.0, "C")
inlet_stream.setPressure(50.0, "bara")

print("Inlet Stream initialized.")

Inlet Stream initialized.


In [150]:
# 3. Configure the TurboExpanderCompressor

# Initialize the unit with the inlet stream
turbo_expander = jneqsim.process.equipment.expander.TurboExpanderCompressor("TurboExpander", inlet_stream)

# Set Design Parameters
turbo_expander.setImpellerDiameter(0.424)
turbo_expander.setDesignSpeed(6850.0)
turbo_expander.setExpanderDesignIsentropicEfficiency(0.88)
turbo_expander.setDesignUC(0.7)
turbo_expander.setDesignQn(0.03328)
turbo_expander.setExpanderOutPressure(35.0) # Set expander outlet pressur ein bara
turbo_expander.setCompressorDesignPolytropicEfficiency(0.81)
turbo_expander.setCompressorDesignPolytropicHead(20.47)
turbo_expander.setMaximumIGVArea(1.637e4)

# Set Reference Curves
# UC vs Efficiency Curve
uc_values = [0.9964751359624449, 0.7590835113213541, 0.984295619176559, 0.8827799803397821, 0.9552460269880922, 1.0]
eff_values = [0.984090909090909, 0.796590909090909, 0.9931818181818183, 0.9363636363636364, 0.9943181818181818, 1.0]
turbo_expander.setUCcurve(uc_values, eff_values)

# Q/N vs Efficiency Curve
qn_eff_x = [0.5, 0.7, 0.85, 1.0, 1.2, 1.4, 1.6]
qn_eff_y = [0.88, 0.91, 0.95, 1.0, 0.97, 0.85, 0.6]
turbo_expander.setQNEfficiencycurve(qn_eff_x, qn_eff_y)

# Q/N vs Head Curve
qn_head_x = [0.5, 0.8, 1.0, 1.2, 1.4, 1.6]
qn_head_y = [1.1, 1.05, 1.0, 0.9, 0.7, 0.4]
turbo_expander.setQNHeadcurve(qn_head_x, qn_head_y)

print("TurboExpanderCompressor configured.")

TurboExpanderCompressor configured.


In [151]:
# 4. Run the Simulation
print("Running simulation...")
turbo_expander.run()
print("Simulation complete.")

ngl_scrubber = jneqsim.process.equipment.separator.Separator("NGLScrubber")
ngl_scrubber.addStream(turbo_expander.getExpanderOutletStream())

outstream = ngl_scrubber.getGasOutStream().clone()

recycle1 = jneqsim.process.equipment.util.Recycle("Recycle1")
recycle1.addStream(ngl_scrubber.getGasOutStream())
recycle1.setOutletStream(outstream)

turbo_expander.setCompressorFeedStream(outstream)

tex_process_1 = jneqsim.process.processmodel.ProcessSystem('tex process')
tex_process_1.add([inlet_stream, turbo_expander, ngl_scrubber, outstream, recycle1])
tex_process_1.run()

# 5. Analyze Results
expander_power = turbo_expander.getPowerExpander("MW")
compressor_power = turbo_expander.getPowerCompressor("MW")
speed = turbo_expander.getSpeed()

print(f"Expander Power: {expander_power:.4f} MW")
print(f"Compressor Power: {compressor_power:.4f} MW")
print(f"Shaft Speed: {speed:.2f} rpm")

# Get Efficiencies
expander_efficiency = turbo_expander.getExpanderIsentropicEfficiency()
compressor_efficiency = turbo_expander.getCompressorPolytropicEfficieny()

print(f"Expander Isentropic Efficiency: {expander_efficiency:.4f}")
print(f"Compressor Polytropic Efficiency: {compressor_efficiency:.4f}")

print('IGV opening ' , turbo_expander.getIGVopening()*100, '%')

# Get Outlet Streams
expander_out_stream = turbo_expander.getExpanderOutletStream()
compressor_out_stream = turbo_expander.getCompressorOutletStream()

print(f"Expander Outlet Temperature: {expander_out_stream.getTemperature('C'):.2f} C")
print(f"Compressor Outlet Temperature: {compressor_out_stream.getTemperature('C'):.2f} C")

print(f"Expander Outlet Pressure: {expander_out_stream.getPressure('bara'):.2f} bara")
print(f"Compressor Outlet Pressure: {compressor_out_stream.getPressure('bara'):.2f} bara")

print('Expander feed gas flow ', expander_out_stream.getFlowRate('MSm3/day'), 'MSm3/day')
print('Compressor gas flow ', compressor_out_stream.getFlowRate('MSm3/day'), 'MSm3/day')

Running simulation...
Simulation complete.
Expander Power: 4.0227 MW
Compressor Power: 4.0227 MW
Shaft Speed: 7382.41 rpm
Expander Isentropic Efficiency: 0.8654
Compressor Polytropic Efficiency: 0.7808
IGV opening  90.55596903048209 %
Expander Outlet Temperature: -14.06 C
Compressor Outlet Temperature: 4.64 C
Expander Outlet Pressure: 35.00 bara
Compressor Outlet Pressure: 44.47 bara
Expander feed gas flow  15.000000000000002 MSm3/day
Compressor gas flow  14.088888064330558 MSm3/day


Print conditions in ngl_scrubber

In [152]:
from neqsim.thermo import printFrame
printFrame(ngl_scrubber.getFluid())

| 0                    | 1          | 2                   | 3                   | 4   | 5   | 6               |
|:---------------------|:-----------|:--------------------|:--------------------|:----|:----|:----------------|
|                      | total      | GAS                 | OIL                 |     |     |                 |
| methane              | 8.57143E-1 | 8.97511E-1          | 2.32916E-1          |     |     | [mole fraction] |
| ethane               | 4.7619E-2  | 4.58906E-2          | 7.43474E-2          |     |     | [mole fraction] |
| propane              | 4.7619E-2  | 3.625E-2            | 2.23424E-1          |     |     | [mole fraction] |
| n-butane             | 4.7619E-2  | 2.03487E-2          | 4.69313E-1          |     |     | [mole fraction] |
|                      |            |                     |                     |     |     |                 |
| Density              |            | 3.52008E1           | 5.36199E2           |     |     | kg/m3     